In [1]:
import chainer

/Users/choumori/project/chainer/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/choumori/project/chainer/lib/python3.6/site-packages/chainer/_environment_check.py:38: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BLAS libraries such as OpenBLAS.
For details of the issue, please see
https://docs.chainer.org/en/stable/tips.html#mnist-example-does-not-converge-in-cpu-mode-on-mac-os-x.

Also note that Chainer does not officially support Mac OS X.
Please use it at your own risk.

  ''')  # NOQA


In [2]:
import chainer.functions as F

In [3]:
import chainer.links as L

In [4]:
from chainer import training, datasets, iterators, optimizers

In [5]:
from chainer.training import extensions

In [6]:
import numpy as np

In [18]:
batch_size =200

In [19]:
uses_device =-1 

In [20]:
class NMIST_Conv_NN(chainer.Chain):

        def __init__(self):
                super(NMIST_Conv_NN, self).__init__()
                with self.init_scope():
                        self.conv1 = L.Convolution2D(1, 8, ksize=3)     # フィルタサイズ＝3で出力数8
                        self.linear1 = L.Linear(1352, 10)                       # 出力数10

        def __call__(self, x, t=None, train=True):
                # 畳み込みニューラルネットワークによる画像認識
                h1 = self.conv1(x)                              # 畳み込み層
                h2 = F.relu(h1)                                 # 活性化関数
                h3 = F.max_pooling_2d(h2, 2)# プーリング層
                h4 = self.linear1(h3)                   # 全結合層
                # 損失か結果を返す
                return F.softmax_cross_entropy(h4, t) if train else F.softmax(h4)

In [21]:
# ニューラルネットワークを作成
model = NMIST_Conv_NN()

In [22]:
if uses_device >= 0:
        # GPUを使う
        chainer.cuda.get_device_from_id(0).use()
        chainer.cuda.check_cuda_available()
        # GPU用データ形式に変換
        model.to_gpu()

In [23]:
# MNISTデータセットを用意する
train, test = chainer.datasets.get_mnist(ndim=3)

In [24]:
# 繰り返し条件を作成する
train_iter = iterators.SerialIterator(train, batch_size, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)

In [25]:
# 誤差逆伝播法アルゴリズムを選択する
optimizer = optimizers.Adam()
optimizer.setup(model)

In [26]:
# デバイスを選択してTrainerを作成する
updater = training.StandardUpdater(train_iter, optimizer, device=uses_device)
trainer = training.Trainer(updater, (5, 'epoch'), out="result")

In [27]:
# テストをTrainerに設定する
trainer.extend(extensions.Evaluator(test_iter, model, device=uses_device))

In [28]:
# 学習の進展を表示するようにする
trainer.extend(extensions.ProgressBar())

In [29]:
# 機械学習を実行する
trainer.run()

     total [###...............................................]  6.67%
this epoch [################..................................] 33.33%
       100 iter, 0 epoch / 5 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [######............................................] 13.33%
this epoch [#################################.................] 66.67%
       200 iter, 0 epoch / 5 epochs
    23.366 iters/sec. Estimated time to finish: 0:00:55.636916.
     total [##########........................................] 20.00%
this epoch [..................................................]  0.00%
       300 iter, 1 epoch / 5 epochs
    20.046 iters/sec. Estimated time to finish: 0:00:59.862096.
     total [#############.....................................] 26.67%
this epoch [################..................................] 33.33%
       400 iter, 1 epoch / 5 epochs
     21.05 iters/sec. Estimated time to finish: 0:00:52.255977.
     total [################...........

In [21]:
# 学習結果を保存する
chainer.serializers.save_hdf5( 'chapt02local.hdf5', model )

RuntimeError: h5py is not installed on your environment.
Please install h5py to activate hdf5 serializers.

  $ pip install h5py

# 結果を見る

In [12]:
if uses_device >= 0:
    import cupy as cp
else:
    cp = np

In [13]:
from PIL import Image

In [14]:
# 学習結果を読み込む
chainer.serializers.load_hdf5( 'mnist1.hdf5', model )

In [15]:
# 画像を読み込む
image = Image.open('test/mnist-0.png').convert('L')

In [16]:
# ニューラルネットワークの入力に合わせて成形する
pixels = cp.asarray(image).astype(cp.float32).reshape(1,1,28,28)
pixels = pixels / 255

In [17]:
# ニューラルネットワークを実行する
result = model(pixels, train=False)

In [18]:
# 実行結果を表示する
for i in range(len(result.data[0])):
    print( str(i) + '\t' + str(result.data[0][i]) )

0	0.99998474
1	2.1946712e-14
2	1.530782e-05
3	6.5191255e-12
4	1.5229287e-15
5	5.7731657e-13
6	1.3733917e-08
7	3.741634e-12
8	8.273766e-09
9	4.029267e-09
